<a href="https://colab.research.google.com/github/elkinkazan/itis-2025-ai-tools/blob/main/Task6_FakeAnalysisPipeline_Iliasova.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:


# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
!pip install --upgrade gspread pandas google-auth
import pandas as pd
import gspread
from google.colab import auth
from google.auth import default
from IPython.display import display
import random
# Authenticate and create the PyDrive client.
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

In [1]:
# FILL THIS
student_name = "ELVIRA ILIASOVA"
group_id = "11-451"

In [ ]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
SPREADSHEET_URL = "https://docs.google.com/spreadsheets/d/1Kfxj2eDFl7xQnXw7Fpb9bwghc65o8xf--VpNxrdWHaY/edit?gid=0#gid=0"
sh = gc.open_by_url(SPREADSHEET_URL)
worksheet = sh.sheet1

# Ensure header row exists
if not worksheet.get_all_values():
    worksheet.append_row(["Student Name", "Group","TaskID", "Score"])


# MAIN NOTEBOOK GOES HERE
task_id = "Task6_FakeAnalysisPipeline"
score = 0
max_score = 25

In [2]:
# 🎓 Домашнее задание: Проверка подлинности видео с использованием видео- и аудиопотока

In [3]:
# 0. Установка зависимостей
!pip install ffmpeg-python resemblyzer scikit-learn tensorflow matplotlib tqdm librosa opencv-python-headless

In [4]:
!pip install gdown
!gdown --folder "https://drive.google.com/drive/folders/1YyoZ2lUWGtmnAo3EPJcvmtCDBvCX8NjY"
!gdown --folder "https://drive.google.com/drive/folders/1yN1fmxLxCP79Lup5IRLuwOaGbi7hPjDV"

Retrieving folder contents
Processing file 1jGxkEO0Mo480qiEnXFca8UOEDq2it7Xl video1.mp4
Processing file 1rpWIef8PeG_0gXF7UwqaedJ89pJJvAqg video2.mp4
Processing file 1KMtcEUdokLJWfi9y0kL1Kap70SVqh56S video3.mp4
Processing file 1Qb8yVwXWrCdjYOrmg0j97v86GSES7jLs video4.mp4
Processing file 1C_Gqtj5Tt7LEjTXt7VTILQjOdFfPpoVa video5.mp4
Processing file 1yXeRjPKMyMm8TAAtpH5CR2vp16Y1A_EZ video6.mp4
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1jGxkEO0Mo480qiEnXFca8UOEDq2it7Xl
To: /content/video_hw/video1.mp4
100% 1.64M/1.64M [00:00<00:00, 128MB/s]
Downloading...
From: https://drive.google.com/uc?id=1rpWIef8PeG_0gXF7UwqaedJ89pJJvAqg
To: /content/video_hw/video2.mp4
100% 31.3M/31.3M [00:00<00:00, 141MB/s]
Downloading...
From: https://drive.google.com/uc?id=1KMtcEUdokLJWfi9y0kL1Kap70SVqh56S
To: /content/video_hw/video3.mp4
100% 50.9M/50.9M [00:00<00:00, 76.7MB/s]
Downloading...
From: htt

In [5]:
# 1. Импорты
import ffmpeg
import os
import librosa
import numpy as np
from resemblyzer import preprocess_wav, VoiceEncoder
from pathlib import Path
from tensorflow.keras.models import load_model
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import glob

In [6]:
# Загрузка обученной модели на LA
model = load_model("keras_hw/audio_detection.keras")  # обучается отдельно
encoder = VoiceEncoder()

# Путь к папке с видео
VIDEO_DIR = "video_hw"
VIDEO_LIST = [os.path.join(VIDEO_DIR, f"video{i}.mp4") for i in range(1, 7)]

Loaded the voice encoder model on cpu in 0.04 seconds.


In [7]:
# Извлечение аудио из видео
def extract_audio_from_video(video_path, output_wav):
    (
        ffmpeg
        .input(video_path)
        .output(output_wav, ac=1, ar=16000)
        .overwrite_output()
        .run(quiet=True)
    )


In [8]:
from pathlib import Path
from resemblyzer import preprocess_wav
# Предсказание с помощью модели
def predict_audio_model(file_path):
    wav = preprocess_wav(Path(file_path))
    # YOU CODE HERE USING THE LA model and resembler predict the video
    # Получение эмбеддинга из голосового аудио
    embed = encoder.embed_utterance(wav)  # encoder должен быть глобальным или передан извне

    # Предсказание модели
    prediction = model.predict(embed.reshape(1, -1))[0][0]
    return prediction, prediction >= 0.5

In [9]:
def extract_simple_features(file_path):
    y, sr = librosa.load(file_path, sr=None)

    # YOUR CODE STARTS HERE
    mfcc = librosa.feature.mfcc(y=y, sr=sr, n_mfcc=20)
    mfcc_std = np.std(mfcc, axis=1)  # std по времени, для каждого коэффициента
    zcr = librosa.feature.zero_crossing_rate(y)[0]  # [0] — извлекаем сам массив
    centroid = librosa.feature.spectral_centroid(y=y, sr=sr)[0]
    # YOUR CODE ENDS HERE

    return {
        "mfcc_std_mean": np.mean(mfcc_std),
        "zcr_mean": np.mean(zcr),
        "centroid_mean": np.mean(centroid)
    }


In [10]:
def simple_fake_audio_detector(file_path):
    f = extract_simple_features(file_path)
    is_fake = f["mfcc_std_mean"] < 25 or f["zcr_mean"] > 0.1 or f["centroid_mean"] > 3500
    return f, is_fake

In [11]:
!pip install protobuf==3.20.3
!pip install mediapipe==0.10.9

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 32.5 MB/s eta 0:00:00


In [12]:
import os
import numpy as np
import cv2
import mediapipe as mp
import tensorflow as tf
from keras.models import load_model

# Отключаем CUDA, если нужно (хотя mediapipe не требует)
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

# Константы
IMG_SIZE = 299
MODEL_PATH = 'keras_hw/model_trained.keras'

# Загрузка модели
model_exception = load_model(MODEL_PATH)

# Настройка mediapipe
mp_face_detection = mp.solutions.face_detection
face_detection = mp_face_detection.FaceDetection(model_selection=1, min_detection_confidence=0.5)

# Преобразование для модели
def preprocess_face(face):
    face = cv2.resize(face, (IMG_SIZE, IMG_SIZE))
    face = face.astype('float32') / 255.0
    return np.expand_dims(face, axis=0)

# Извлечение лица через mediapipe
def extract_face(frame):
    results = face_detection.process(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    if not results.detections:
        return None
    bbox = results.detections[0].location_data.relative_bounding_box
    h, w, _ = frame.shape
    x1 = int(bbox.xmin * w)
    y1 = int(bbox.ymin * h)
    x2 = int((bbox.xmin + bbox.width) * w)
    y2 = int((bbox.ymin + bbox.height) * h)

    # Проверка на выход за границы
    x1, y1 = max(0, x1), max(0, y1)
    x2, y2 = min(w, x2), min(h, y2)

    return frame[y1:y2, x1:x2]

# Анализ видеопотока
def analyze_video_stream(video_path, max_frames=30):
    cap = cv2.VideoCapture(video_path)
    predictions = []
    frame_count = 0

    while frame_count < max_frames:
        ret, frame = cap.read()
        if not ret:
            break
        face = extract_face(frame)
        if face is None:
            continue
        input_face = preprocess_face(face)
        pred = model_exception.predict(input_face, verbose=0)[0][0]
        predictions.append(pred)
        frame_count += 1

    cap.release()

    if predictions:
        return "Fake detected", np.mean(predictions) > 0.5
    else:
        return None, False


In [16]:

# Основной анализ одного видеофайла

def analyze_video_file(video_path):
    print(f"\n📂 Файл: {os.path.basename(video_path)}")
    audio_path = video_path.replace(".mp4", ".wav")
    extract_audio_from_video(video_path, audio_path)
    visual_log, visual_flag = analyze_video_stream(video_path)
    model_score, model_label = predict_audio_model(audio_path)
    heuristics, heur_label = simple_fake_audio_detector(audio_path)

    print("🎥 Видео (по кадрам):", visual_log, "⛔" if visual_flag else "✅")
    print("🔊 Аудио (Resemblyzer+NN):", "Fake" if model_label else "Real", f"(score={model_score:.3f})")
    print("🔎 Аудио (эвристика):", "Fake" if heur_label else "Real", heuristics)

    return {
        "video": os.path.basename(video_path),
        "visual_flag": int(visual_flag),
        "nn_result": int(model_label),
        "heur_result": int(heur_label),
        "model_score": float(model_score),
        "heuristics": heuristics
    }




In [17]:
VIDEO_LIST

['video_hw/video1.mp4',
 'video_hw/video2.mp4',
 'video_hw/video3.mp4',
 'video_hw/video4.mp4',
 'video_hw/video5.mp4',
 'video_hw/video6.mp4']

In [18]:
# Запуск пайплайна на всех видео
results = []
score = 0
import ffmpeg

for video_path in VIDEO_LIST:
    print(video_path)
    result = analyze_video_file(video_path)
    results.append(result)

# Тест структуры результатов

def test_pipeline_structure(results):
    global score
    assert len(results) == 6, "Должно быть 6 видео"
    for r in results:
        assert 'nn_result' in r and 'heur_result' in r and 'visual_flag' in r
        score += 5
        assert isinstance(r['model_score'], float)
        score += 10
        assert isinstance(r['heuristics'], dict)
        score += 10
    print("✅ Pipeline test passed.")

test_pipeline_structure(results)

video_hw/video1.mp4

📂 Файл: video1.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
🎥 Видео (по кадрам): Fake detected ✅
🔊 Аудио (Resemblyzer+NN): Real (score=0.000)
🔎 Аудио (эвристика): Fake {'mfcc_std_mean': 14.544644, 'zcr_mean': 0.087081179792065, 'centroid_mean': 1292.4313741279502}
video_hw/video2.mp4

📂 Файл: video2.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
🎥 Видео (по кадрам): Fake detected ✅
🔊 Аудио (Resemblyzer+NN): Real (score=0.001)
🔎 Аудио (эвристика): Fake {'mfcc_std_mean': 17.455591, 'zcr_mean': 0.09279700170589399, 'centroid_mean': 1242.533605577194}
video_hw/video3.mp4

📂 Файл: video3.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 46ms/step
🎥 Видео (по кадрам): Fake detected ✅
🔊 Аудио (Resemblyzer+NN): Real (score=0.000)
🔎 Аудио (эвристика): Fake {'mfcc_std_mean': 16.525593, 'zcr_mean': 0.09569876085904068, 'centroid_mean': 1293.226942948503}
video_hw/video4.mp4

📂 Файл: video4.mp4
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step
🎥 Видео (по кадрам): Fake detected ✅
🔊 Аудио (Resemblyzer+NN): Real (sc

In [ ]:
# DO NOT MODIFY  !!!
# DO NOT EXECUTE !!!
# Save the result to Google Sheets
from datetime import datetime

# Get current date and time
now = datetime.now()
timestamp = now.strftime("%Y-%m-%d %H:%M:%S")
worksheet.append_row([student_name,group_id, task_id, score, timestamp])

print(f"Test completed! {student_name}, your score is {score}/{max_score}.")